In [28]:
import os
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from tensorflow.keras.models import load_model
from sklearn.metrics import classification_report, accuracy_score

# TensorFlow 로그 레벨 설정
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

# GPU 설정
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(f"Failed to set memory growth: {e}")
else:
    print("No GPU devices found. Running on CPU.")


In [29]:
def load_data(directory):
    """
    데이터를 로드하고, 레이블과 파일 이름을 포함하여 반환하는 함수

    Parameters:
        directory (str): 데이터가 저장된 디렉토리 경로

    Returns:
        X (np.ndarray): 입력 데이터
        y (np.ndarray): 레이블
        participants (np.ndarray): 참여자 정보
        file_names (np.ndarray): 각 샘플의 파일 이름
    """
    # Positive(긍정)과 Negative(부정) 데이터를 저장할 리스트 초기화
    files_positive = []
    files_negative = []

    # 디렉토리에 있는 파일들을 탐색
    for file in os.listdir(directory):
        # 파일 이름에 '_positive_'가 포함되어 있으면 positive 파일 리스트에 추가
        if '_positive_' in file:
            files_positive.append(os.path.join(directory, file))
        # 파일 이름에 '_negative_'가 포함되어 있으면 negative 파일 리스트에 추가
        elif '_negative_' in file:
            files_negative.append(os.path.join(directory, file))

    # positive 또는 negative 파일이 하나도 없을 경우 예외를 발생시킴
    if not files_positive or not files_negative:
        raise ValueError(f"No files found. Positive: {files_positive}, Negative: {files_negative}")

    # positive와 negative 파일의 데이터를 읽어서 각각 리스트에 저장
    positive_data = [np.load(file) for file in files_positive]  # positive 파일 데이터 로드
    negative_data = [np.load(file) for file in files_negative]  # negative 파일 데이터 로드

    # positive와 negative 데이터를 합쳐서 X 데이터로 생성
    # axis=0은 데이터를 샘플 축(행)으로 결합함
    X = np.concatenate(positive_data + negative_data, axis=0)

    # y 레이블 생성
    # positive 데이터의 개수만큼 1로 된 레이블 생성 후
    # negative 데이터의 개수만큼 0으로 된 레이블을 추가
    y = np.array([1] * len(np.concatenate(positive_data)) + [0] * len(np.concatenate(negative_data)))

    # 참여자 정보(participants) 및 파일 이름(file_names) 생성
    participants = []
    file_names = []
    for file in files_positive + files_negative:
        # 파일 이름에서 참여자 ID 추출 (파일 이름 예: "001_positive_data.npy")
        participant_id = os.path.basename(file).split('_')[0]
        data = np.load(file)
        num_samples = data.shape[0]
        participants.extend([participant_id] * num_samples)
        file_names.extend([os.path.basename(file)] * num_samples)

    # 리스트를 NumPy 배열로 변환
    participants = np.array(participants)
    file_names = np.array(file_names)

    # 최종적으로 데이터(X), 레이블(y), 참여자 정보(participants), 파일 이름(file_names)를 반환
    return X, y, participants, file_names

In [30]:
def create_cnn_model(input_shape, num_classes):
    """
    CNN 모델을 생성하고 컴파일하는 함수

    Parameters:
        input_shape (tuple): 입력 데이터의 형태 (예: (timesteps, features))
        num_classes (int): 출력 클래스의 수 (예: 감정 레이블의 개수)

    Returns:
        model (Sequential): 생성된 CNN 모델
    """
    # Sequential 모델 생성
    model = Sequential([
        # 1D Convolutional Layer: 32개의 필터, kernel_size=3, 활성화 함수 ReLU
        # input_shape 지정 (첫 번째 레이어에서만 필요)
        Conv1D(32, kernel_size=3, activation='relu', input_shape=input_shape),

        # MaxPooling Layer: 풀링 크기(pool_size)=2, strides=1
        MaxPooling1D(pool_size=2, strides=1),

        # 1D Convolutional Layer: 64개의 필터, kernel_size=3, 활성화 함수 ReLU
        Conv1D(64, kernel_size=3, activation='relu'),

        # MaxPooling Layer: 풀링 크기(pool_size)=2, strides=1
        MaxPooling1D(pool_size=2, strides=1),

        # Flatten Layer: 1D 데이터를 1차원으로 펼침
        Flatten(),

        # Fully Connected Layer: 128개의 뉴런, 활성화 함수 ReLU
        Dense(128, activation='relu'),

        # Dropout Layer: 50%의 뉴런을 무작위로 비활성화 (overfitting 방지)
        Dropout(0.5),

        # Output Layer: 클래스 수에 따라 Softmax 활성화 함수 사용
        Dense(num_classes, activation='softmax')
    ])

    # 모델 컴파일: 최적화 알고리즘, 손실 함수, 평가지표 설정
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model  # 생성된 모델 반환


In [31]:
# Step 3: Load and evaluate saved model
def evaluate_model(X, y, participants, model_path):
    """
    Load a saved model and evaluate it on the test data.

    Parameters:
        X (np.ndarray): Preprocessed feature data.
        y (np.ndarray): Corresponding labels.
        participants (np.ndarray): Participant IDs.
        model_path (str): Path to the saved model.
    """
    model = load_model(model_path)
    all_true, all_pred = [], []
    prediction_details = []

    for participant in np.unique(participants):
        participant_mask = (participants == participant)
        X_test = X[participant_mask]
        y_test = y[participant_mask]

        # Convert labels to categorical
        y_test_categorical = to_categorical(y_test, num_classes=len(np.unique(y)))

        # Predict
        y_pred_prob = model.predict(X_test)
        y_pred = np.argmax(y_pred_prob, axis=1)

        # Collect results
        all_true.extend(y_test)
        all_pred.extend(y_pred)

        # Collect prediction details
        for idx, pred in enumerate(y_pred):
            correctness = "Correct" if pred == y_test[idx] else "Incorrect"
            prediction_details.append(f"Participant {participant}: Predicted={pred}, Actual={y_test[idx]} ({correctness})")

    # Final evaluation
    report = classification_report(all_true, all_pred, digits=4)
    print(report)

    # Save detailed report and predictions
    with open("evaluation_results.txt", "w") as eval_file:
        eval_file.write("Final Accuracy: {:.4f}\n\n".format(np.mean(np.array(all_true) == np.array(all_pred))))
        eval_file.write("Detailed Classification Report:\n")
        eval_file.write(report)
        eval_file.write("\nPrediction Details (file-level):\n")
        eval_file.write("\n".join(prediction_details))

In [32]:
def main():
    """
    메인 함수: CNN 모델 생성, 저장 및 평가 실행
    """
    # 데이터 디렉토리 및 모델 저장 경로 설정
    data_directory = '/home/bcml1/2025_EMOTION/DEAP_EEG/ch_BPF'  # 데이터를 저장한 디렉토리 경로
    model_save_path = 'cnn_intra_model.h5'  # 모델 저장 경로

    # 모델 생성 및 저장
    print("Creating and saving the model...")
    input_shape = (100, 64)  # 예시 입력 데이터 형태 (timesteps, features)
    num_classes = 2  # 예시 클래스 개수
    create_cnn_model(input_shape, num_classes, model_save_path)

    print("Training the model...")
    train_model_intra_session(X, y, participants, model_path, result_path)
    
    # 저장된 모델 평가
    print("\nEvaluating the saved model...")
    evaluate_model(X, y, data_directory, model_save_path)

if __name__ == "__main__":
    main()


/home/bcml1/sigenv/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Creating and saving the model...
Model saved to cnn_intra_model.h5

Evaluating the saved model...


Model loaded from cnn_intra_model.h5

Fold 1:
Original X_test shape: (39, 8, 5120)


ValueError: cannot reshape array of size 1597440 into shape (39,80,64)